# Decision Trees

Leitura dos datasets tratados

In [1]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']

treino = pd.read_csv('../datasets/treinoTratado.csv', na_values=allowed_nans, keep_default_na=False)
teste = pd.read_csv('../datasets/testeTratado.csv', na_values=allowed_nans, keep_default_na=False)


## Construção e modelação

In [2]:
X_treino = treino.drop('Injeçao na rede (kWh)', axis=1)
y_treino = treino['Injeçao na rede (kWh)']

X_teste = teste

With artificial neural networks its mandatory to perform data normalization.

In [3]:
scaler = StandardScaler()

X_treino_normalized = scaler.fit_transform(X_treino)
X_teste_normalized = scaler.fit_transform(X_teste)

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=183, max_iter=2000)

params = {
    'activation': ['relu', 'tanh', 'logistic'],
    'learning_rate_init': [0.05, 0.1, 0.3],  # Learning rate
    'hidden_layer_sizes': [(50,50), (100,100)],
}

grid = GridSearchCV(estimator=clf,
                    param_grid=params,
                    cv=10,
                    n_jobs=5,
                    verbose=2)
grid.fit(X_treino_normalized, y_treino)
grid.best_score_, grid.best_params_

Fitting 10 folds for each of 18 candidates, totalling 180 fits


(0.838095238095238,
 {'activation': 'logistic',
  'hidden_layer_sizes': (100, 100),
  'learning_rate_init': 0.05})

In [6]:
predictions = grid.predict(X_teste_normalized)

predict_data = {'Result': predictions}
df = pd.DataFrame(predict_data)
df.index += 1
df.index.name = 'RowId'

df.to_csv(f'predictions/redesNeuronaisPrediction.csv')